In [1]:
import re
import gc

import pandas as pd
import numpy as np

### dataディレクトリの中に格納されているxlsxファイルのパスを取得

In [2]:
import glob

file_list = glob.glob('data/enavi*.csv')

file_list

['data\\enavi201904(6875).csv',
 'data\\enavi201905(6875).csv',
 'data\\enavi201906(6875).csv',
 'data\\enavi201907(6875).csv',
 'data\\enavi201908(6875).csv',
 'data\\enavi201909(6875).csv',
 'data\\enavi201910(6875).csv',
 'data\\enavi201911(6875).csv',
 'data\\enavi201912(6875).csv',
 'data\\enavi202001(6875).csv',
 'data\\enavi202002(6875).csv',
 'data\\enavi202003(6875).csv',
 'data\\enavi202004(6875).csv',
 'data\\enavi202005(6875).csv',
 'data\\enavi202006(6875).csv']

### データフレームとして1ファイルのみ読み込み

In [20]:
#dtypes={'科目コード':str, '金額':int}
#parse_date = ['決算日']

df = pd.read_csv(file_list[0])

df = df.dropna(subset=['利用日'])

df['test/test'] = 'test'

display(df.shape, df.head(1))

(54, 11)

,利用日,利用店名・商品名,利用者,支払方法,利用金額,支払手数料,支払総額,4月支払金額,5月繰越残高,新規サイン,test/test
0,2019/03/30,AMAZON.CO.JP,本人,1回払い,1782.0,0.0,1782.0,1782.0,0.0,*,test


In [15]:
class ImportDataToSQLServer:
    def __init__(self, database):
        '''
        Input
        ・database：使用するデータベース名(予めSQLserver上で「CREATE DATABASE [データベース名]」というコードを実行させて作成しておく)
        '''
        self.database = database
        self.driver ='{SQL Server}'
        self.server = 'localhost\SQLEXPRESS'
        self.trusted_connection='yes'
        self.file_name_list = ['rcard_明細_201904']
        # 欠損値があればNoneを補完
        self.df_list = [df.where(df.notnull(), None)]
        
    def connect_to_sqlserver(self):
        '''
        関数内容
        ・PythonからSQLserverへ接続する関数

        '''
        import pyodbc
        conn =\
        pyodbc.connect('DRIVER='+self.driver+';SERVER='+self.server+';DATABASE='+self.database+';Trusted_Connection='+self.trusted_connection+';')
        curs = conn.cursor()

        return conn, curs
    
    def extract_colname_and_dtype_from_dataframe(self, df):
        '''
        関数内容
        ・読み込んだデータのカラム名とデータ型を抽出し、それらをデータフレームに格納する関数
        '''
        dtypes_df = pd.DataFrame(df.dtypes).reset_index()
        dtypes_df.columns=['カラム名','データ型']
        
        return dtypes_df
        

    def get_colname_and_dtype_to_create_table(self, df):
        '''
        関数内容
        ・create tableに必要なカラム名とデータ型を取得し、2つを特定の形式に並べた文字列を作成する関数
        '''
        # 読み込んだデータのカラム名とデータ型を抽出し、それらをデータフレームに格納する関数の実行
        df1 = self.extract_colname_and_dtype_from_dataframe(df)


        # SQLServerへのインポート用のデータ型名にリネーム
        df1['データ型'] = np.where(df1['データ型']=='object', 'NVARCHAR(255)',
                                     np.where(df1['データ型']=='float64', 'NUMERIC(25)',df1['データ型']))

        ct_col_list = df1['カラム名'].tolist()
        # カラム名に'/'が含まれているとSQLServerへのインポート時にエラーが発生するのでリネーム
        ct_col_list =['['+ col_name + ']' if '/' in col_name else col_name for col_name in ct_col_list]
        # カラム名先頭が数字の場合もSQLServerへのインポート時にエラーが発生するのでリネーム
        ct_col_list =['['+ col + ']' if re.match('^[0-9]', col) else col for col in ct_col_list ]

        ct_dtype_list = df1['データ型'].tolist()

        combi_ct_list = [str(col_name)+' '+str(dtype_name) for col_name, dtype_name in zip (ct_col_list, ct_dtype_list)]

        colname_and_dtype_str = ','.join(map(str, combi_ct_list))

        return colname_and_dtype_str
    
    def get_colname_and_questionmark_to_insert_data(self, df):
        '''
        関数内容
        ・insert intoに必要なカラム名と?マークを取得する関数
        
        '''
        # 読み込んだデータのカラム名とデータ型を抽出し、それらをデータフレームに格納する関数の実行
        df1 = self.extract_colname_and_dtype_from_dataframe(df)
        
        it_col_list = df1['カラム名'].tolist()
        # カラム名に'/'が含まれているとSQLServerへのインポート時にエラーが発生するのでリネーム
        it_col_list =['['+ col_name + ']' if '/' in col_name else col_name for col_name in it_col_list]
        # カラム名先頭が数字の場合もSQLServerへのインポート時にエラーが発生するのでリネーム
        it_col_list =['['+ col + ']' if re.match('^[0-9]', col) else col for col in it_col_list ]
        
        it_col_str = ','.join(map(str, it_col_list))
        
        questionmark_str = '?,'*len(df1)
        # 最後の「,」は必要ない + ()を追加
        questionmark_str = '(' + questionmark_str[:-1] + ')'
        
        return it_col_str, questionmark_str
        

    def create_tables_in_sqlserver(self, df):
        '''
        関数内容
        ・データベース内にテーブルを作成する関数
        '''
        # create tableに必要なカラム名とデータ型を取得し、2つを特定の形式に並べた文字列を作成する関数を実行
        #colname_and_dtype_str = get_colname_and_dtype_to_create_table(self, df=dtypes_df)
        
        conn, curs = self.connect_to_sqlserver()
        colname_and_dtype_str = self.get_colname_and_dtype_to_create_table(df)

        for file_name in self.file_name_list:
            print(file_name+'テーブルをSQLServerに作成します。')
            print(colname_and_dtype_str)
            curs.execute(
            'CREATE TABLE ' + file_name +\
            '(' + colname_and_dtype_str +')')
            print(file_name+'テーブルをSQLServerに作成し終えました。')

            conn.commit()
            
    def insert_into_sqlserver(self, df):
        '''
        関数内容
        ・データベース内のテーブルにデータを追加する関数
       
        '''
        
        # insert intoに必要なカラム名と?マークを取得する関数を実行
        #it_col_str, questionmark_str = get_colname_and_questionmark_to_insert_data(self, df=dtypes_df)
        
        conn, curs = self.connect_to_sqlserver()
        
        it_col_str, questionmark_str = self.get_colname_and_questionmark_to_insert_data(df)
        
        for file_name, df in zip(self.file_name_list, self.df_list):
            print(file_name+'テーブルにデータを格納します。')
            values_tuple = [tuple(row) for row in df.values]
            # カラムが複数ある結果、挿入したい値が複数ある場合はcurs.executemanyを使う
            curs.executemany(
                'INSERT INTO ' + file_name +'(' + it_col_str + ')'+ 'VALUES ' + questionmark_str ,values_tuple
            )
            print(file_name+'テーブルにデータを格納し終えました。')

            conn.commit()
    
    def disconnect_from_sqlserver(self):
        '''
        関数内容
        ・SQLserverへの接続を切る関数
        '''
        conn, curs = self.connect_to_sqlserver()
        
        curs.close()
        conn.close()

In [16]:
# オブジェクトの作成 
import_to_sql = ImportDataToSQLServer('C10000_database')

In [17]:
# PythonからSQLserverへ接続する関数
import_to_sql.connect_to_sqlserver()

(<pyodbc.Connection at 0x205f8812b90>, <pyodbc.Cursor at 0x205f7db1378>)

In [10]:
# 読み込んだデータのカラム名とデータ型を抽出し、それらをデータフレームに格納
col_dtype_df = import_to_sql.extract_colname_and_dtype_from_dataframe(df)

# データベース内にテーブルを作成
import_to_sql.create_tables_in_sqlserver(df)

rcard_明細_201904テーブルをSQLServerに作成します。
利用日 NVARCHAR(255),利用店名・商品名 NVARCHAR(255),利用者 NVARCHAR(255),支払方法 NVARCHAR(255),利用金額 NUMERIC(25),支払手数料 NUMERIC(25),支払総額 NUMERIC(25),[4月支払金額] NUMERIC(25),[5月繰越残高] NUMERIC(25),新規サイン NVARCHAR(255),[test/test] NVARCHAR(255)
rcard_明細_201904テーブルをSQLServerに作成し終えました。


In [11]:
# データベース内のテーブルにデータを追加
import_to_sql.insert_into_sqlserver(df)

rcard_明細_201904テーブルにデータを格納します。
rcard_明細_201904テーブルにデータを格納し終えました。


In [18]:
# SQLserverへの接続を切る
import_to_sql.disconnect_from_sqlserver()

In [19]:
gc.collect()

446

参考

https://qiita.com/gaborotta/items/3f2f2fd492163a1ec007

https://qiita.com/YoshitakaOkada/items/d6d97c5954adae148b7a